# 400 Train the StyleGAN2-ADA ##

The StyleGAN2 requires (from the README.md)
- Python 3.7
- CUDA GPU
- PyTorch 1.7.1 (torch)
- CUDA toolkit 11.0 or later
- Python libraries: click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In my experiment, I select runtime GPU Tesla T4 for running.

In [ ]:
# Check for a GPU
!nvidia-smi

# Check for Python version
!python --version

Sun Jul  2 06:57:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Remark:**
The Colab runtime is now running with Python 3.10.x which is newer than the StyleGAN2 spec

In [ ]:
# Check for the required libraries
!pip list | grep 'torch\|click\|requests\|tqdm\|pyspng\|ninja\|imageio-ffmpeg'

# Expected output
# click                         8.1.3
# imageio-ffmpeg                0.4.8
# requests                      2.27.1
# requests-oauthlib             1.3.1
# requests-unixsocket           0.2.0
# torch                         2.0.1+cu118
# torchaudio                    2.0.2+cu118
# torchdata                     0.6.1
# torchsummary                  1.5.1
# torchtext                     0.15.2
# torchvision                   0.15.2+cu118
# tqdm                          4.65.0

click                            8.1.3
click-plugins                    1.1.1
imageio-ffmpeg                   0.4.8
requests                         2.27.1
requests-oauthlib                1.3.1
requests-unixsocket              0.2.0
torch                            2.0.1+cu118
torchaudio                       2.0.2+cu118
torchdata                        0.6.1
torchsummary                     1.5.1
torchtext                        0.15.2
torchvision                      0.15.2+cu118
tqdm                             4.65.0


**Remark:** The runtime does not have `pyspng` and `ninja` so we must install it.

In [ ]:
!pip install pyspng ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 18.9 MB/s eta 0:00:00


In [ ]:
# Recheck again
!pip list | grep 'torch\|click\|requests\|tqdm\|pyspng\|ninja\|imageio-ffmpeg'

# Expected output
# click                         8.1.3
# imageio-ffmpeg                0.4.8
# ninja                         1.11.1
# pyspng                        0.1.1
# requests                      2.27.1
# requests-oauthlib             1.3.1
# requests-unixsocket           0.2.0
# torch                         2.0.1+cu118
# torchaudio                    2.0.2+cu118
# torchdata                     0.6.1
# torchsummary                  1.5.1
# torchtext                     0.15.2
# torchvision                   0.15.2+cu118
# tqdm                          4.65.0

click                            8.1.3
click-plugins                    1.1.1
imageio-ffmpeg                   0.4.8
ninja                            1.11.1
pyspng                           0.1.1
requests                         2.27.1
requests-oauthlib                1.3.1
requests-unixsocket              0.2.0
torch                            2.0.1+cu118
torchaudio                       2.0.2+cu118
torchdata                        0.6.1
torchsummary                     1.5.1
torchtext                        0.15.2
torchvision                      0.15.2+cu118
tqdm                             4.65.0


## Clone Git Repository ##

We will use the forked version of the stylegan2-ada-pytorch that supports grayscale image generation in branch `greyscale-images-support`.

https://github.com/tanutapi/stylegan2-ada-pytorch


In [ ]:
!rm -rf stylegan2-ada-pytorch/
!git clone --branch greyscale-images-support https://github.com/tanutapi/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 165 (delta 19), reused 21 (delta 9), pack-reused 132
Receiving objects: 100% (165/165), 1.13 MiB | 2.56 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
!python ./stylegan2-ada-pytorch/train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in
  the paper "Training Generative Adversarial
  Networks with Limited Data".

  Examples:

  # Train with custom dataset using 1 GPU.
  python train.py --outdir=~/training-runs --data=~/mydataset.zip --gpus=1

  # Train class-conditional CIFAR-10 using 2 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/cifar10.zip \
      --gpus=2 --cfg=cifar --cond=1

  # Transfer learn MetFaces from FFHQ using 4 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/metfaces.zip \
      --gpus=4 --cfg=paper1024 --mirror=1 --resume=ffhq1024 --snap=10

  # Reproduce original StyleGAN2 config F.
  python train.py --outdir=~/training-runs --data=~/datasets/ffhq.zip \
      --gpus=8 --cfg=stylegan2 --mirror=1 --aug=noaug

  Base configs (--cfg):
    auto       Automatically select reasonable defaults based on resolution
               and GPU count. Good starting point for new datasets.
    stylegan2  R

In [ ]:
SEED = 2

# RESUME = False
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00000-4500_dataset_tool_stylegan2-ada-auto1/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00001-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00002-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00003-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00004-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00005-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00006-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00007-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00008-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00009-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00010-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00011-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00012-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00013-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00014-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'
# RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00015-4500_dataset_tool_stylegan2-ada-auto1-kimg600-resumecustom/network-snapshot-000600.pkl'

RESUME = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500/00000-4500_dataset_tool_stylegan2-ada-auto1-kimg600/network-snapshot-000000.pkl'

DATA_PATH = f'/content/drive/MyDrive/Datasets/SOCOFing/splits/seed_{SEED}/4500_dataset_tool_stylegan2-ada.zip'

OUTPUT_PATH = f'/content/drive/MyDrive/Datasets/GANs/stylegan2-ada-socofing-seed{SEED}-4500'

if RESUME:
  !python ./stylegan2-ada-pytorch/train.py --resume={RESUME} --data={DATA_PATH} --outdir={OUTPUT_PATH} --gpus=1 --cfg=auto --snap=50 --seed=0 --kimg=600
else:
  !python ./stylegan2-ada-pytorch/train.py --data={DATA_PATH} --outdir={OUTPUT_PATH} --gpus=1 --cfg=auto --snap=50 --seed=0 --kimg=600


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 50,
  "network_snapshot_ticks": 50,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/Datasets/SOCOFing/splits/seed_2/4500_dataset_tool_stylegan2-ada.zip",
    "use_labels": false,
    "max_size": 4500,
    "xflip": false,
    "resolution": 128
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_grou

In [ ]:
from google.colab import drive
drive.flush_and_unmount()